In [ ]:
# import statsmodels.api as sm
# from statsmodels.stats.multitest import multipletests as mpt
# import numpy as np
import pandas as pd
# from matplotlib import pyplot as plt
# import itertools
# import seaborn as sns
# %pylab inline

In [ ]:
import numpy

In [ ]:
'''
preparing dataframes (best to do this on your data):
ct_freq should be a df of cell type frequencies per patient

all_freqs should have cols patient, neighborhood, cell types, giving
the frequency of cells of that type in that neighborhood in that patient
'''
ct_freq = pd.read_csv('cell_type_freqs_per_patient')
all_freqs = pd.read_csv('all_freqs')

#you will need to set the following with your own data
neighborhood_col = 'neighborhood10'
nbs = [0,2,3,4,5,6,7,8,9]
patients = range(1,36) 
group = pd.Series([0]*15+ [1]*20) 
cells = ['B cells','T cells']

In [3]:
def normalize(X):
    arr = np.array(X.fillna(0).values)
    return pd.DataFrame(np.log2(1e-3 + arr/arr.sum(axis =1, keepdims = True)), index = X.index.values, columns = X.columns).fillna(0)


In [ ]:
# data prep
# normalized overall cell type frequencies
X_cts = normalize(ct_frq.reset_index().set_index('patients').loc[patients,cells])

# normalized neighborhood specific cell type frequencies
df_list = []

for nb in nbs:
    cond_nb = all_freqs.loc[all_freqs[neighborhood_col]==nb,cells].rename({col: col+'_'+str(nb) for col in cells},axis = 1).set_index('patients')
    df_list.append(normalize(cond_nb))

X_cond_nb = pd.concat(df_list,axis = 1).loc[patients]

In [ ]:
#differential enrichment for all cell subsets
changes = {}
nbs =[0, 2, 3, 4, 6, 7, 8, 9]
for col in cells:
    for nb in nbs:
        #build a design matrix with a constant, group 0 or 1 and the overall frequencies
        X = pd.concat([X_cts[col], group.astype('int'),pd.Series(np.ones(len(group)), index = group.index.values)],axis = 1).values
        if col+'_%d'%nb in X_condnb_all.columns:
            #set the neighborhood specific ct freqs as the outcome
            Y = X_cond_nb[col+'_%d'%nb].values
            X = X[~pd.isna(Y)]
            Y = Y[~pd.isna(Y)]
            #fit a linear regression model
            results = sm.OLS(Y,X).fit()
            #find the params and pvalues for the group coefficient
            changes[(col,nb)] = (results.pvalues[1], results.params[1])
        

#make a dataframe with coeffs and pvalues
dat = (pd.DataFrame(changes).loc[1].unstack())
dat = pd.DataFrame(np.nan_to_num(dat.values),index = dat.index, columns = dat.columns).T.sort_index(ascending=True).loc[:,X_cts.columns]
pvals = (pd.DataFrame(changes).loc[0].unstack()).T.sort_index(ascending=True).loc[:,X_cts.columns]

#this is where you should correct pvalues for multiple testing 


#plot as heatmap
f, ax = plt.subplots(figsize = (20,10))
g = sns.heatmap(dat,cmap = 'bwr', vmin = -1, vmax = 1,cbar=False,ax = ax)
for a,b in zip(*np.where (pvals<0.05)):
    plt.text(b+.5,a+.55,'*',fontsize = 20,ha = 'center',va = 'center')
plt.tight_layout()
